**Задача:**

Выполнить оценку работы библиотеки по извлечению текста новости. Необходимо определить полноту извлекаемого текста и оценить значимость тех частей текста, которые выделить не удалось. Проанализировать и предложить изменения в алгоритм работы библиотеки.



Желаемый результат:

Заполненная таблица -

- с эталоном текста для каждой новости;
- с результатами оценки полноты (сравнения эталона текста и текста, извлеченного библиотекой);
- оценка значимости текста, который выделить не удалось;
- комментарии, если есть предложения по улучшению.


In [30]:
pip install trafilatura

In [31]:
pip install rouge

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from difflib import unified_diff
from rouge import Rouge
import trafilatura

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import recall_score

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [33]:
from google.colab import drive # подключение гугл диска с файлами
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
df = pd.read_excel('/content/drive/My Drive/Kaspersky/Test_check.xlsx')
df.head()

,URL,lib_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-otmena-sanktsiy-ssha/,"Серьезнее других от введения западных санкций пострадали отрасли, зависимые от импорта, — это не..."
1,https://ria.ru/20250311/klyuchevaya_stavka-1962773695.html,"МОСКВА, 14 фев — РИА Новости Ключевая процентная ставка — важнейший индикатор состояния экономик..."
2,https://expert.ru/mnenie/denis-manturov-gosudarstvo-v-ramkakh-natsproektov-obespechivaet-prefere...,"Со следующего года начнется реализация 12 мегапроектов, направленных на достижение технологическ..."
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность для россиян обезопасить себя от мошеннических займ...
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые упаковки товаров или надписи об их экологичности и п..."


In [35]:
df.shape

(50, 2)

Датасет загружен и открыт, имеет две колонки:

1. URL - Ссылка на новость
2. lib_text - Эталонный текст.

In [36]:
def extract_news_text(url):
    """Извлекает текст новости с помощью trafilatura."""
    try:
        downloaded = trafilatura.fetch_url(url)
        text = trafilatura.extract(downloaded)
        return text if text else "Не удалось извлечь текст"
    except Exception as e:
        return f"Ошибка: {e}"

In [37]:
# Применяем trafilatura к каждой ссылке
df["extracted_text"] = df["URL"].apply(
    lambda x: extract_news_text(x) if x else "Некорректный URL"
)

In [52]:
pd.set_option('display.max_colwidth', 100)
df.head()

,URL,lib_text,extracted_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-otmena-sanktsiy-ssha/,"Серьезнее других от введения западных санкций пострадали отрасли, зависимые от импорта, — это не...",Какие отрасли пострадали сильнее\nСерьезнее других от введения западных санкций пострадали отрас...
1,https://ria.ru/20250311/klyuchevaya_stavka-1962773695.html,"МОСКВА, 14 фев — РИА Новости Ключевая процентная ставка — важнейший индикатор состояния экономик...","14:39 15.05.2025\n(обновлено: 14:40 15.05.2025)\nКлючевая ставка ЦБ РФ: для чего нужна, почему м..."
2,https://expert.ru/mnenie/denis-manturov-gosudarstvo-v-ramkakh-natsproektov-obespechivaet-prefere...,"Со следующего года начнется реализация 12 мегапроектов, направленных на достижение технологическ...","Со следующего года начнется реализация 12 мегапроектов, направленных на достижение технологическ..."
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность для россиян обезопасить себя от мошеннических займ...,"11:27 30.04.2025\n(обновлено: 00:02 01.05.2025)\nСамозапрет на кредиты: суть, как оформить, плюс..."
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые упаковки товаров или надписи об их экологичности и п...","Заходя в магазин, вы часто замечали зеленые упаковки товаров или надписи об их экологичности и п..."


In [39]:
print('\033[1m Эталонный текст\033[0m')
df['lib_text'][1]

 Эталонный текст


'МОСКВА, 14 фев — РИА Новости Ключевая процентная ставка — важнейший индикатор состояния экономики и финансового рынка России. Ключевая ставка ЦБ РФ на сегодня составляет 21% годовых. Совет директоров Банка России 20 декабря 2024 года принял решение сохранить ключевую ставку на уровне 21,00% годовых, так как текущее инфляционное давление остается достаточно высоким. Этот инструмент позволяет Центробанку регулировать экономическую активность и уровень инфляции в стране. Что это такое простыми словами, причины и последствия ее повышения для банков и бизнеса, график заседаний ЦБ РФ по ключевой ставке на 2025 год, ее показатели сегодня, прогноз на ближайшие годы и мнения экспертов, – в материале РИА Новости.\nКлючевая ставка Центрального банка Российской Федерации представляет собой основной инструмент денежно-кредитной политики, который используется для регулирования экономической активности и уровня инфляции в стране. Это наименьший процент, под который Центробанк выдает кредиты коммерче

In [40]:
print('\033[1m Извлеченный текст\033[0m')
df['extracted_text'][1]

 Извлеченный текст


'14:39 15.05.2025\n(обновлено: 14:40 15.05.2025)\nКлючевая ставка ЦБ РФ: для чего нужна, почему меняется и на что влияет\n© Инфографика\nЧитать ria.ru в\nМОСКВА, 25 апр — РИА Новости. Ключевая процентная ставка — важнейший индикатор состояния экономики и финансового рынка России. Ключевая ставка ЦБ РФ на сегодня составляет 21% годовых. Совет директоров Банка России 25 апреля 2025 года принял решение сохранить ключевую ставку на уровне 21,00% годовых, так как текущее инфляционное давление остается достаточно высоким. Этот инструмент позволяет Центробанку регулировать экономическую активность и уровень инфляции в стране. Что это такое простыми словами, причины и последствия ее повышения для банков и бизнеса, график заседаний ЦБ РФ по ключевой ставке на 2025 год, ее показатели сегодня, прогноз на ближайшие годы и мнения экспертов, – в материале РИА Новости.\nКлючевая ставка\nКлючевая ставка Центрального банка Российской Федерации представляет собой основной инструмент денежно-кредитной по

**Выберем метрики:**

ROUGE-Recall показывает, насколько полно был извлечён текст.

missing_word_count — сколько слов потеряно.

significant_missing — потерянные важные слова (длинные или именованные).

In [41]:
rouge = Rouge()

def analyze_row(row):
    reference = row['lib_text']
    extracted = row['extracted_text']

    # ROUGE — базовая метрика полноты и совпадения по фразам
    scores = rouge.get_scores(extracted, reference)[0]
    rouge_recall = scores['rouge-l']['r']


    # Найдём потерянные слова (которые есть в reference, но отсутствуют в extracted)
    ref_words = set(word_tokenize(reference.lower()))
    ext_words = set(word_tokenize(extracted.lower()))
    missing_words = ref_words - ext_words

    # Оценим значимость пропущенных слов: количество сущностей или часто встречающихся слов
    significant_missing = [w for w in missing_words if len(w) > 1 or w[0].isupper()]

    return pd.Series({
        "rouge_recall": rouge_recall,
        "missing_word_count": len(missing_words),
        "significant_missing": ", ".join(significant_missing[:10])
    })

In [42]:
# Применяем анализ по строкам
results = df.apply(analyze_row, axis=1)

# Объединяем с исходными данными
df_analysis = pd.concat([df, results], axis=1)

# Сохраняем результат
df_analysis.to_excel("analyzed_results.xlsx", index=False)

In [47]:
pd.set_option('display.max_colwidth', 100)
df_analysis.head()

,URL,lib_text,extracted_text,rouge_recall,missing_word_count,significant_missing
0,https://expert.ru/ekonomika/vygodna-li-rossii-otmena-sanktsiy-ssha/,"Серьезнее других от введения западных санкций пострадали отрасли, зависимые от импорта, — это не...",Какие отрасли пострадали сильнее\nСерьезнее других от введения западных санкций пострадали отрас...,1.000000,0,
1,https://ria.ru/20250311/klyuchevaya_stavka-1962773695.html,"МОСКВА, 14 фев — РИА Новости Ключевая процентная ставка — важнейший индикатор состояния экономик...","14:39 15.05.2025\n(обновлено: 14:40 15.05.2025)\nКлючевая ставка ЦБ РФ: для чего нужна, почему м...",0.989574,10,"–говорит, сейчас, центробанка., последних, траектории, неоднозначны, 2024года, дальнейшей, фев, ..."
2,https://expert.ru/mnenie/denis-manturov-gosudarstvo-v-ramkakh-natsproektov-obespechivaet-prefere...,"Со следующего года начнется реализация 12 мегапроектов, направленных на достижение технологическ...","Со следующего года начнется реализация 12 мегапроектов, направленных на достижение технологическ...",1.000000,0,
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность для россиян обезопасить себя от мошеннических займ...,"11:27 30.04.2025\n(обновлено: 00:02 01.05.2025)\nСамозапрет на кредиты: суть, как оформить, плюс...",1.000000,0,
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые упаковки товаров или надписи об их экологичности и п...","Заходя в магазин, вы часто замечали зеленые упаковки товаров или надписи об их экологичности и п...",1.000000,0,


Посмотрим новости, которые извлечены не полностью

In [48]:
df_significant_missing = df_analysis.query('missing_word_count != 0')
df_significant_missing

,URL,lib_text,extracted_text,rouge_recall,missing_word_count,significant_missing
1,https://ria.ru/20250311/klyuchevaya_stavka-1962773695.html,"МОСКВА, 14 фев — РИА Новости Ключевая процентная ставка — важнейший индикатор состояния экономик...","14:39 15.05.2025\n(обновлено: 14:40 15.05.2025)\nКлючевая ставка ЦБ РФ: для чего нужна, почему м...",0.989574,10,"–говорит, сейчас, центробанка., последних, траектории, неоднозначны, 2024года, дальнейшей, фев, ..."
15,https://www.vesti.ru/article/4393501,Сегодня мы рассказываем о полуторагодовалой Лере Смирновой из Хабаровска.\nУ девочки в первые дн...,Сегодня мы рассказываем о полуторагодовалой Лере Смирновой из Хабаровска.\nУ девочки в первые дн...,0.975610,7,"681, 528, 340, 22,236, 073, 965"
33,https://ria.ru/20241220/shtrafy-1990507458.html,"МОСКВА, 20 дек — РИА Новости. Штрафы ГИБДД за нарушение правил дорожного движения призваны повыс...",https://ria.ru/20250401/shtrafy-1990507458.html\nНовые штрафы ГИБДД: что изменится с 1 апреля 20...,0.977134,27,"свежему, 01.01.2025., в., 750., дек, они, 1500-2250., этот, 10000., законопроекту"


In [49]:
# Показывать до 1000 символов в каждой строке
pd.set_option('display.max_colwidth', 1000)

# Выводим столбец
print('\033[1m Неизвлеченные слова:\033[0m \n')
print(df_significant_missing['significant_missing'])

 Неизвлеченные слова: 

1     –говорит, сейчас, центробанка., последних, траектории, неоднозначны, 2024года, дальнейшей, фев, уточнят
15                                                                            681, 528, 340, 22,236, 073, 965
33                          свежему, 01.01.2025., в., 750., дек, они, 1500-2250., этот, 10000., законопроекту
Name: significant_missing, dtype: object


In [51]:
# Итоговая таблица:
pd.set_option('display.max_colwidth', 100)
df_analysis.head()

,URL,lib_text,extracted_text,rouge_recall,missing_word_count,significant_missing
0,https://expert.ru/ekonomika/vygodna-li-rossii-otmena-sanktsiy-ssha/,"Серьезнее других от введения западных санкций пострадали отрасли, зависимые от импорта, — это не...",Какие отрасли пострадали сильнее\nСерьезнее других от введения западных санкций пострадали отрас...,1.000000,0,
1,https://ria.ru/20250311/klyuchevaya_stavka-1962773695.html,"МОСКВА, 14 фев — РИА Новости Ключевая процентная ставка — важнейший индикатор состояния экономик...","14:39 15.05.2025\n(обновлено: 14:40 15.05.2025)\nКлючевая ставка ЦБ РФ: для чего нужна, почему м...",0.989574,10,"–говорит, сейчас, центробанка., последних, траектории, неоднозначны, 2024года, дальнейшей, фев, ..."
2,https://expert.ru/mnenie/denis-manturov-gosudarstvo-v-ramkakh-natsproektov-obespechivaet-prefere...,"Со следующего года начнется реализация 12 мегапроектов, направленных на достижение технологическ...","Со следующего года начнется реализация 12 мегапроектов, направленных на достижение технологическ...",1.000000,0,
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность для россиян обезопасить себя от мошеннических займ...,"11:27 30.04.2025\n(обновлено: 00:02 01.05.2025)\nСамозапрет на кредиты: суть, как оформить, плюс...",1.000000,0,
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые упаковки товаров или надписи об их экологичности и п...","Заходя в магазин, вы часто замечали зеленые упаковки товаров или надписи об их экологичности и п...",1.000000,0,


In [54]:
df_analysis.to_excel("/content/drive/My Drive/Kaspersky//analyzed_results.xlsx", index=False)

**Выводы о работе trafilatura**

1. Неизвлекаются ключевые числовые данные - высокая значимость

    Важные цифры (например, «10000», «750.», «1500-2250.») не были извлечены.
    Это может быть связано с тем, что trafilatura может игнорировать таблицы, числовые списки.

2. Проблемы с датами и аббревиатурами - высокая значимость

    фев, дек, 01.01.2025. и подобные — не всегда попадают в основной текст при очистке.Возможно, даты в исходной разметке были в мета-тегах, подписях или сносках, которые игнорируются.

3. Экономические/законодательные термины - высокая значимость

    центробанка., законопроекту., траектории, дальнейшей, уточнят	ключевые сущности и действия

4. Проблемы с частями цитат - средняя значимость

    Слова типа –говорит, они, этот, сейчас указывают, что могла быть усечена или не извлечена часть цитаты.

**Предложения по улучшению**

1. Извлекать числа отдельно от текста.
2. Заменить библиотеку для извлечения новости.